Notebook to create a Nemo Bathymetry file for the ERDDAP server  
Based on:   
Nancy/NEMO depths vs bathymetry file.ipynb

In [10]:
import netCDF4 as nc
import numpy as np 
import matplotlib.pyplot as plt

from salishsea_tools import nc_tools

%matplotlib inline

In [2]:
mesh = nc.Dataset('/data/nsoontie/MEOPAR/NEMO-forcing/grid/mesh_mask_SalishSea2.nc')
mbathy = mesh.variables['mbathy'][0,:,:] 
#used to calculate number of vertical ocean grid cells at each (i,j) (1=land point)
gdepw = mesh.variables['gdepw'][0,:,:,:]
surface_tmask = mesh.variables['tmask'][0,0,:,:]
surface_tmask = np.abs(surface_tmask-1)

In [4]:
NEMO_bathy = np.zeros(mbathy.shape)
for i in np.arange(NEMO_bathy.shape[1]):
    for j in np.arange(NEMO_bathy.shape[0]):
        level = mbathy[j,i]
        NEMO_bathy[j,i] = gdepw[level,j,i]
NEMO_bathy = np.ma.masked_array(NEMO_bathy, mask = surface_tmask)


In [8]:
lats = mesh.variables['nav_lat'][:]
lons = mesh.variables['nav_lon'][:]

In [13]:
# build nc file
new_bathy = nc.Dataset('SalishSea2_NEMO_bathy.nc', 'w')
nc_tools.init_dataset_attrs(
    new_bathy, 
    title='Bathymetry after NEMO Processes, SalishSea Bathymetry 2', 
    notebook_name='NEMOBathymetryfromMeshMask', 
    nc_filepath='NEMO-forcing/grid/SalishSea2_NEMO_bathy.nc',
    comment='Bathymetry, Latitudes and Longitudes')
new_bathy.createDimension('y', 898)
new_bathy.createDimension('x', 398)
nc_tools.show_dimensions(new_bathy)
# variables
latitude = new_bathy.createVariable('latitude', 'float32', ('y','x'))
latitude.long_name = 'Latitude'
latitude.units = 'degrees_north'
latitude[:] = lats
longitude = new_bathy.createVariable('longitude', 'float32', ('y','x'))
longitude.long_name = 'Longitude'
longitude.units = 'degrees_east'
longitude[:] = lons

bathymetry = new_bathy.createVariable('bathymetry', 'float32', ('y','x'))
bathymetry.units = 'm'
bathymetry.long_name = 'Depth of Bottom'  
bathymetry.coordinates = 'longitude latitude'
bathymetry.grid = 'Salish Sea 2'
bathymetry[:] = NEMO_bathy

new_bathy.history = """[2016-02-05 Created]"""

file format: NETCDF4
Conventions: CF-1.6
title: Bathymetry after NEMO Processes, SalishSea Bathymetry 2
institution: Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia
source: https://bitbucket.org/salishsea/analysis/src/tip/Susan/NEMOBathymetryfromMeshMask.ipynb
references: REQUIRED
history: [2016-02-06 16:35:19] Created netCDF4 zlib=True dataset.
comment: Bathymetry, Latitudes and Longitudes
<class 'netCDF4._netCDF4.Dimension'>: name = 'y', size = 898

<class 'netCDF4._netCDF4.Dimension'>: name = 'x', size = 398



In [14]:
new_bathy.close()